In [3]:
# 下载数据集
!wget https://ascend-professional-construction-dataset.obs.myhuaweicloud.com/deep-learning/aclImdb_v1.tar.gz
# 下载模型
!wget https://ascend-professional-construction-dataset.obs.myhuaweicloud.com/deep-learning/glove.zip
!unzip glove.zip

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'unzip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     -------------------------------------- 24.0/24.0 MB 395.6 kB/s eta 0:00:00
     ------------------------------------ 983.8/983.8 kB 356.0 kB/s eta 0:00:00
     -------------------------------------- 58.6/58.6 kB 443.9 kB/s eta 0:00:00


In [ ]:
!wget https://ascend-professional-construction-dataset.obs.myhuaweicloud.com/deep-learning/aclImdb_v1.tar.gz

尝试改变每层神经元数量，但是发现过拟合明显

In [ ]:
#################################################
##################依赖库导入######################
#################################################
import os
import struct

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print("依赖库导入成功")
#################################################
##################参数配置########################
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
epochs_list = [10,20,50]
batch_size_list = [256,512,1024]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0

conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv3 = layers.Conv2D(128, (3,3), activation='relu')
max_pool_1 = layers.MaxPooling2D((2,2))
max_pool_2 = layers.MaxPooling2D((4,4))
max_pool_3 = layers.MaxPooling2D((8,8))
#max_pool_list = [max_pool_1, max_pool_2]
max_pool_list = [(2,2)]
flat_layer = layers.Flatten()
fc = layers.Dense(128, activation='relu')
output = layers.Dense(10, 'softmax')

drop_1 = keras.layers.Dropout(0.2)
drop_2 = keras.layers.Dropout(0.2)
drop_3 = keras.layers.Dropout(0.2)
drop_4 = keras.layers.Dropout(0.5)

conv1_new = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28,28,1) )
BN=keras.layers.BatchNormalization()
conv1_new_conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv1_new_conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv1_new_conv3 = layers.Conv2D(128, (3,3), activation='relu')
conv1_new_max_pool_1 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_2 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_3 = layers.MaxPooling2D((2,2))
conv1_new_flat_layer = layers.Flatten()
conv1_new_fc = layers.Dense(128, activation='relu')
conv1_new_output = layers.Dense(10, 'softmax')

print("参数配置成功")
#################################################
##################训练开始########################
#################################################
# TensorFlow Keras 使用Keras Sequential API
for i in range(len(epochs_list)):
    for j in range(len(batch_size_list)):
        for k in range(len(max_pool_list)):
            print("##########################################################################")
            print("开始训练模型，epochs为"+str(epochs_list[i])+", batch_size为"+str(batch_size_list[j])+", max_pool为"+str(max_pool_list[k]))
            model = models.Sequential()
            #if k == 1:
            #    model.summary()
            model.add(conv1)
            model.add(conv2)
            model.add(conv3)
            model.add(layers.MaxPooling2D(max_pool_list[k]))
            model.add(flat_layer)
            model.add(fc)
            model.add(output)

            model.summary()
            model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            ################################################
            model.fit(train_images_norm, train_labels, epochs=epochs_list[i], batch_size=batch_size_list[j], shuffle=True, validation_split=0.1)
            ################################################
            test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            
            new_model = models.Sequential()

            new_model.add(conv1_new_conv1)
            #第一卷积层'conv1'没有池化层和Dropout层
            new_model.add(conv1_new_conv2)
            new_model.add(conv1_new_max_pool_2)
            new_model.add(drop_2)
            new_model.add(conv1_new_conv3)
            new_model.add(conv1_new_max_pool_3)
            new_model.add(drop_3)
            new_model.add(conv1_new_flat_layer)
            new_model.add(conv1_new_fc)
            new_model.add(BN)
            new_model.add(drop_4)
            new_model.add(conv1_new_output)

            new_model.summary()
            new_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            new_model.fit(train_images_norm, train_labels, epochs=30, batch_size=512, shuffle=True, validation_split=0.1)
            test_loss, test_accuracy = new_model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            print(test_loss)
            print(new_model.history.history.keys())
            #plt.subplot(3*3*1,3,3*i+j+1)
            plt.figure(3*i+j+1)
            plt.plot(new_model.history.history['accuracy'])
            plt.plot(new_model.history.history['val_accuracy'])
            plt.title("Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            #plt.subplot(3*3*1,3,3*i+j+1+1)
            plt.figure(3*i+j+2)
            plt.plot(new_model.history.history['loss'])
            plt.plot(new_model.history.history['val_loss'])
            plt.title("Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            ############模型预测#################
#             predicted_test_labels = new_model.predict(test_images_norm)
#             print(predicted_test_labels.shape)
#             print(predicted_test_labels[88])


#             predicted_test_labels_index = np.argmax(predicted_test_labels[88])

#             print(predicted_test_labels_index)
#             print(test_labels[88])

#             #plt.subplot(3*3*1,3,3*i+j+2+1)
#             plt.figure(3*i+j+3)
#             plt.figure()
#             plt.imshow(np.squeeze(test_images[88]))
#             plt.show()
            plt.figure(figsize=(10,10))
            for key in range(5):
                number = random.randint(0,10000)
                print("number is "+str(number))
                plt.subplot(1,5,key+1)
                predicted_test_labels = new_model.predict(test_images_norm)
                print(predicted_test_labels.shape)
                print(predicted_test_labels[number])

                predicted_test_labels_index = np.argmax(predicted_test_labels[number])

                print(predicted_test_labels_index)
                print(test_labels[number])

                plt.imshow(np.squeeze(test_images[number]))
            plt.show()
print("模型不同参数配置成功")

发现轮回次数如果从20轮开始，在已经测试的batchsize下过拟合现象很明显，所以接下来测试一下降低epoch以及batchsize的效果。但是从测试集里面随机取5个进行识别分类发现准确率也还行，可能是“5”这个样本量太小了，偶然性太大

In [ ]:
#################################################
##################依赖库导入######################
#################################################
import os
import struct

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print("依赖库导入成功")
#################################################
##################参数配置########################
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
epochs_list = [3,5,10]
batch_size_list = [128,256,512]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0

conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv3 = layers.Conv2D(128, (3,3), activation='relu')
max_pool_1 = layers.MaxPooling2D((2,2))
max_pool_2 = layers.MaxPooling2D((4,4))
max_pool_3 = layers.MaxPooling2D((8,8))
#max_pool_list = [max_pool_1, max_pool_2]
max_pool_list = [(2,2)]
flat_layer = layers.Flatten()
fc = layers.Dense(128, activation='relu')
output = layers.Dense(10, 'softmax')

drop_1 = keras.layers.Dropout(0.2)
drop_2 = keras.layers.Dropout(0.2)
drop_3 = keras.layers.Dropout(0.2)
drop_4 = keras.layers.Dropout(0.5)

conv1_new = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28,28,1) )
BN=keras.layers.BatchNormalization()
conv1_new_conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv1_new_conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv1_new_conv3 = layers.Conv2D(128, (3,3), activation='relu')
conv1_new_max_pool_1 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_2 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_3 = layers.MaxPooling2D((2,2))
conv1_new_flat_layer = layers.Flatten()
conv1_new_fc = layers.Dense(128, activation='relu')
conv1_new_output = layers.Dense(10, 'softmax')

print("参数配置成功")
#################################################
##################训练开始########################
#################################################
# TensorFlow Keras 使用Keras Sequential API
for i in range(len(epochs_list)):
    for j in range(len(batch_size_list)):
        for k in range(len(max_pool_list)):
            print("##########################################################################")
            print("开始训练模型，epochs为"+str(epochs_list[i])+", batch_size为"+str(batch_size_list[j])+", max_pool为"+str(max_pool_list[k]))
            model = models.Sequential()
            #if k == 1:
            #    model.summary()
            model.add(conv1)
            model.add(conv2)
            model.add(conv3)
            model.add(layers.MaxPooling2D(max_pool_list[k]))
            model.add(flat_layer)
            model.add(fc)
            model.add(output)

            model.summary()
            model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            ################################################
            model.fit(train_images_norm, train_labels, epochs=epochs_list[i], batch_size=batch_size_list[j], shuffle=True, validation_split=0.1)
            ################################################
            test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            
            new_model = models.Sequential()

            new_model.add(conv1_new_conv1)
            #第一卷积层'conv1'没有池化层和Dropout层
            new_model.add(conv1_new_conv2)
            new_model.add(conv1_new_max_pool_2)
            new_model.add(drop_2)
            new_model.add(conv1_new_conv3)
            new_model.add(conv1_new_max_pool_3)
            new_model.add(drop_3)
            new_model.add(conv1_new_flat_layer)
            new_model.add(conv1_new_fc)
            new_model.add(BN)
            new_model.add(drop_4)
            new_model.add(conv1_new_output)

            new_model.summary()
            new_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            new_model.fit(train_images_norm, train_labels, epochs=30, batch_size=512, shuffle=True, validation_split=0.1)
            test_loss, test_accuracy = new_model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            print(test_loss)
            print(new_model.history.history.keys())
            #plt.subplot(3*3*1,3,3*i+j+1)
            plt.figure(3*i+j+1)
            plt.plot(new_model.history.history['accuracy'])
            plt.plot(new_model.history.history['val_accuracy'])
            plt.title("Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            #plt.subplot(3*3*1,3,3*i+j+1+1)
            plt.figure(3*i+j+2)
            plt.plot(new_model.history.history['loss'])
            plt.plot(new_model.history.history['val_loss'])
            plt.title("Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            ############模型预测#################
#             predicted_test_labels = new_model.predict(test_images_norm)
#             print(predicted_test_labels.shape)
#             print(predicted_test_labels[88])


#             predicted_test_labels_index = np.argmax(predicted_test_labels[88])

#             print(predicted_test_labels_index)
#             print(test_labels[88])

#             #plt.subplot(3*3*1,3,3*i+j+2+1)
#             plt.figure(3*i+j+3)
#             plt.figure()
#             plt.imshow(np.squeeze(test_images[88]))
#             plt.show()
            plt.figure(figsize=(10,10))
            for key in range(5):
                number = random.randint(0,10000)
                print("number is "+str(number))
                plt.subplot(1,5,key+1)
                predicted_test_labels = new_model.predict(test_images_norm)
                print(predicted_test_labels.shape)
                print(predicted_test_labels[number])

                predicted_test_labels_index = np.argmax(predicted_test_labels[number])

                print(predicted_test_labels_index)
                print(test_labels[number])

                plt.imshow(np.squeeze(test_images[number]))
            plt.show()
print("模型不同参数配置成功")

从5轮开始已经是一直过拟合了，10轮的测试没有意义，直接停止测试。发现3轮，batchsize设置128的效果还行，在此基础上重新开始修改神经元数量的实验

In [ ]:
#################################################
##################依赖库导入######################
#################################################
import os
import struct
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print("依赖库导入成功")
#################################################
##################参数配置########################
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
epochs_list = [3]
batch_size_list = [128]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0
conv_list = [[32,64,128],[64,64,128],[32,32,128]]
conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv3 = layers.Conv2D(128, (3,3), activation='relu')
max_pool_1 = layers.MaxPooling2D((2,2))
max_pool_2 = layers.MaxPooling2D((4,4))
max_pool_3 = layers.MaxPooling2D((8,8))
max_pool_list = [max_pool_1, max_pool_3, max_pool_3]
flat_layer = layers.Flatten()
fc = layers.Dense(128, activation='relu')
output = layers.Dense(10, 'softmax')

drop_1 = keras.layers.Dropout(0.2)
drop_2 = keras.layers.Dropout(0.2)
drop_3 = keras.layers.Dropout(0.2)
drop_4 = keras.layers.Dropout(0.5)

conv1_new = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28,28,1) )
BN=keras.layers.BatchNormalization()
conv1_new_conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv1_new_conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv1_new_conv3 = layers.Conv2D(128, (3,3), activation='relu')
conv1_new_max_pool_1 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_2 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_3 = layers.MaxPooling2D((2,2))
conv1_new_flat_layer = layers.Flatten()
conv1_new_fc = layers.Dense(128, activation='relu')
conv1_new_output = layers.Dense(10, 'softmax')

print("参数配置成功")
#################################################
##################训练开始########################
#################################################
# TensorFlow Keras 使用Keras Sequential API
for i in range(len(epochs_list)):
    for j in range(len(batch_size_list)):
        for k in range(len(conv_list)):
            print("##########################################################################")
            print("开始训练模型，epochs为"+str(epochs_list[i])+", batch_size为"+str(batch_size_list[j])+", Conv2D为"+str(conv_list[k]))
            model = models.Sequential()

            model.add(layers.Conv2D(conv_list[k][0], (3,3), activation='relu', input_shape=(28,28,1) ))
            model.add(layers.Conv2D(conv_list[k][1], (3,3), activation='relu'))
            model.add(layers.Conv2D(conv_list[k][2], (3,3), activation='relu'))
            model.add(max_pool_1)
            model.add(flat_layer)
            model.add(fc)
            model.add(output)

            model.summary()
            model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            ################################################
            model.fit(train_images_norm, train_labels, epochs=epochs_list[i], batch_size=batch_size_list[j], shuffle=True, validation_split=0.1)
            ################################################
            test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)





            new_model = models.Sequential()

            new_model.add(conv1_new_conv1)
            #第一卷积层'conv1'没有池化层和Dropout层
            new_model.add(conv1_new_conv2)
            new_model.add(conv1_new_max_pool_2)
            new_model.add(drop_2)
            new_model.add(conv1_new_conv3)
            new_model.add(conv1_new_max_pool_3)
            new_model.add(drop_3)
            new_model.add(conv1_new_flat_layer)
            new_model.add(conv1_new_fc)
            new_model.add(BN)
            new_model.add(drop_4)
            new_model.add(conv1_new_output)

            new_model.summary()
            new_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            new_model.fit(train_images_norm, train_labels, epochs=30, batch_size=512, shuffle=True, validation_split=0.1)
            test_loss, test_accuracy = new_model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            print(test_loss)
            print(new_model.history.history.keys())
            
            #plt.subplot(3,3,3*k+1)
            print("k="+str(k))
            plt.figure(3*k+1)
            plt.plot(new_model.history.history['accuracy'])
            plt.plot(new_model.history.history['val_accuracy'])
            plt.title("Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            #plt.subplot(3,3,3*k+2)
            plt.figure(3*k+2)
            plt.plot(new_model.history.history['loss'])
            plt.plot(new_model.history.history['val_loss'])
            plt.title("Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            ############模型预测#################
#             for key in range(5)
#             predicted_test_labels = new_model.predict(test_images_norm)
#             print(predicted_test_labels.shape)
#             print(predicted_test_labels[88])


#             predicted_test_labels_index = np.argmax(predicted_test_labels[88])

#             print(predicted_test_labels_index)
#             print(test_labels[88])

            #plt.subplot(3,3,3*k+3)
            # plt.figure(3*k+3)
            # plt.figure()
            # plt.imshow(np.squeeze(test_images[88]))
            # plt.show()
            plt.figure(figsize=(10,10))
            for key in range(5):
                number = random.randint(0,10000)
                print("number is "+str(number))
                plt.subplot(1,5,key+1)
                predicted_test_labels = new_model.predict(test_images_norm)
                print(predicted_test_labels.shape)
                print(predicted_test_labels[number])


                predicted_test_labels_index = np.argmax(predicted_test_labels[number])

                print(predicted_test_labels_index)
                print(test_labels[number])

                #plt.subplot(3,3,3*k+3)
                #plt.figure(3*k+3)
                #plt.figure()
                plt.imshow(np.squeeze(test_images[number]))
            plt.show()
print("模型不同参数配置成功")

对正则化参数进行调试，前边模型参数仍然采用实验手册上参数

In [ ]:
#################################################
##################依赖库导入######################
#################################################
import os
import struct
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print("依赖库导入成功")
#################################################
##################参数配置########################
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
epochs_list = [5,10,20,50]
batch_size_list = [64,128,256,512]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0
conv_list =  [[32,64,128]]
conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv3 = layers.Conv2D(128, (3,3), activation='relu')
max_pool_1 = layers.MaxPooling2D((2,2))
max_pool_2 = layers.MaxPooling2D((4,4))
max_pool_3 = layers.MaxPooling2D((8,8))
max_pool_list = [max_pool_1, max_pool_3, max_pool_3]
flat_layer = layers.Flatten()
fc = layers.Dense(128, activation='relu')
output = layers.Dense(10, 'softmax')

drop_1 = keras.layers.Dropout(0.2)
drop_2 = keras.layers.Dropout(0.2)
drop_3 = keras.layers.Dropout(0.2)
drop_4 = keras.layers.Dropout(0.5)

conv1_new = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28,28,1) )
BN=keras.layers.BatchNormalization()
conv1_new_conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv1_new_conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv1_new_conv3 = layers.Conv2D(128, (3,3), activation='relu')
conv1_new_max_pool_1 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_2 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_3 = layers.MaxPooling2D((2,2))
conv1_new_flat_layer = layers.Flatten()
conv1_new_fc = layers.Dense(128, activation='relu')
conv1_new_output = layers.Dense(10, 'softmax')

print("参数配置成功")
#################################################
##################训练开始########################
#################################################
# TensorFlow Keras 使用Keras Sequential API
for i in range(len(epochs_list)):
    for j in range(len(batch_size_list)):
        for k in range(len(conv_list)):
            print("##########################################################################")
            print("正则化模型，epochs为"+str(epochs_list[i])+", batch_size为"+str(batch_size_list[j])+", Conv2D为"+str(conv_list[k]))
            model = models.Sequential()

            model.add(conv1)
            model.add(conv2)
            model.add(conv3)
            model.add(max_pool_1)
            model.add(flat_layer)
            model.add(fc)
            model.add(output)

            model.summary()
            model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            ################################################
            model.fit(train_images_norm, train_labels, epochs=20, batch_size=512, shuffle=True, validation_split=0.1)
            ################################################
            test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)





            new_model = models.Sequential()

            new_model.add(layers.Conv2D(conv_list[k][0], (3,3), activation='relu', input_shape=(28,28,1) ))
            #第一卷积层'conv1'没有池化层和Dropout层
            new_model.add(layers.Conv2D(conv_list[k][1], (3,3), activation='relu'))
            new_model.add(conv1_new_max_pool_2)
            new_model.add(drop_2)
            new_model.add(layers.Conv2D(conv_list[k][2], (3,3), activation='relu'))
            new_model.add(conv1_new_max_pool_3)
            new_model.add(drop_3)
            new_model.add(conv1_new_flat_layer)
            new_model.add(conv1_new_fc)
            new_model.add(BN)
            new_model.add(drop_4)
            new_model.add(conv1_new_output)

            new_model.summary()
            new_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            new_model.fit(train_images_norm, train_labels, epochs=epochs_list[i], batch_size=batch_size_list[j], shuffle=True, validation_split=0.1)
            test_loss, test_accuracy = new_model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            print(test_loss)
            print(new_model.history.history.keys())
            
            #plt.subplot(3,3,3*k+1)
            print("k="+str(k))
            plt.figure(3*k+1)
            plt.plot(new_model.history.history['accuracy'])
            plt.plot(new_model.history.history['val_accuracy'])
            plt.title("Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            #plt.subplot(3,3,3*k+2)
            plt.figure(3*k+2)
            plt.plot(new_model.history.history['loss'])
            plt.plot(new_model.history.history['val_loss'])
            plt.title("Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            ############模型预测#################
#             for key in range(5)
#             predicted_test_labels = new_model.predict(test_images_norm)
#             print(predicted_test_labels.shape)
#             print(predicted_test_labels[88])


#             predicted_test_labels_index = np.argmax(predicted_test_labels[88])

#             print(predicted_test_labels_index)
#             print(test_labels[88])

            #plt.subplot(3,3,3*k+3)
            # plt.figure(3*k+3)
            # plt.figure()
            # plt.imshow(np.squeeze(test_images[88]))
            # plt.show()
            plt.figure(figsize=(10,10))
            for key in range(5):
                number = random.randint(0,10000)
                print("number is "+str(number))
                plt.subplot(1,5,key+1)
                predicted_test_labels = new_model.predict(test_images_norm)
                print(predicted_test_labels.shape)
                print(predicted_test_labels[number])


                predicted_test_labels_index = np.argmax(predicted_test_labels[number])

                print(predicted_test_labels_index)
                print(test_labels[number])

                #plt.subplot(3,3,3*k+3)
                #plt.figure(3*k+3)
                #plt.figure()
                plt.imshow(np.squeeze(test_images[number]))
            plt.show()
print("模型不同参数配置成功")

In [ ]:
#################################################
##################依赖库导入######################
#################################################
import os
import struct
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print("依赖库导入成功")
#################################################
##################参数配置########################
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
epochs_list = [20]
batch_size_list = [256]
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0
conv_list =  [[32,64,128],[64,64,128],[32,32,128]]
conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv3 = layers.Conv2D(128, (3,3), activation='relu')
max_pool_1 = layers.MaxPooling2D((2,2))
max_pool_2 = layers.MaxPooling2D((4,4))
max_pool_3 = layers.MaxPooling2D((8,8))
max_pool_list = [max_pool_1, max_pool_3, max_pool_3]
flat_layer = layers.Flatten()
fc = layers.Dense(128, activation='relu')
output = layers.Dense(10, 'softmax')

drop_1 = keras.layers.Dropout(0.2)
drop_2 = keras.layers.Dropout(0.2)
drop_3 = keras.layers.Dropout(0.2)
drop_4 = keras.layers.Dropout(0.5)

conv1_new = layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28,28,1) )
BN=keras.layers.BatchNormalization()
conv1_new_conv1 = layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1) )
conv1_new_conv2 = layers.Conv2D(64, (3,3), activation='relu')
conv1_new_conv3 = layers.Conv2D(128, (3,3), activation='relu')
conv1_new_max_pool_1 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_2 = layers.MaxPooling2D((2,2))
conv1_new_max_pool_3 = layers.MaxPooling2D((2,2))
conv1_new_flat_layer = layers.Flatten()
conv1_new_fc = layers.Dense(128, activation='relu')
conv1_new_output = layers.Dense(10, 'softmax')

print("参数配置成功")
#################################################
##################训练开始########################
#################################################
# TensorFlow Keras 使用Keras Sequential API
for i in range(len(epochs_list)):
    for j in range(len(batch_size_list)):
        for k in range(len(conv_list)):
            print("##########################################################################")
            print("正则化模型，epochs为"+str(epochs_list[i])+", batch_size为"+str(batch_size_list[j])+", Conv2D为"+str(conv_list[k]))
            model = models.Sequential()

            model.add(conv1)
            model.add(conv2)
            model.add(conv3)
            model.add(max_pool_1)
            model.add(flat_layer)
            model.add(fc)
            model.add(output)

            model.summary()
            model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            ################################################
            model.fit(train_images_norm, train_labels, epochs=20, batch_size=512, shuffle=True, validation_split=0.1)
            ################################################
            test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)





            new_model = models.Sequential()

            new_model.add(layers.Conv2D(conv_list[k][0], (3,3), activation='relu', input_shape=(28,28,1) ))
            #第一卷积层'conv1'没有池化层和Dropout层
            new_model.add(layers.Conv2D(conv_list[k][1], (3,3), activation='relu'))
            new_model.add(conv1_new_max_pool_2)
            new_model.add(drop_2)
            new_model.add(layers.Conv2D(conv_list[k][2], (3,3), activation='relu'))
            new_model.add(conv1_new_max_pool_3)
            new_model.add(drop_3)
            new_model.add(conv1_new_flat_layer)
            new_model.add(conv1_new_fc)
            new_model.add(BN)
            new_model.add(drop_4)
            new_model.add(conv1_new_output)

            new_model.summary()
            new_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

            new_model.fit(train_images_norm, train_labels, epochs=epochs_list[i], batch_size=batch_size_list[j], shuffle=True, validation_split=0.1)
            test_loss, test_accuracy = new_model.evaluate(test_images_norm, test_labels)
            print(test_accuracy)
            print(test_loss)
            print(new_model.history.history.keys())
            
            #plt.subplot(3,3,3*k+1)
            print("k="+str(k))
            plt.figure(3*k+1)
            plt.plot(new_model.history.history['accuracy'])
            plt.plot(new_model.history.history['val_accuracy'])
            plt.title("Accuracy")
            plt.ylabel("accuracy")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            #plt.subplot(3,3,3*k+2)
            plt.figure(3*k+2)
            plt.plot(new_model.history.history['loss'])
            plt.plot(new_model.history.history['val_loss'])
            plt.title("Loss")
            plt.ylabel("loss")
            plt.xlabel("epoch")
            plt.legend(['train', 'test'], loc='upper left')
            plt.show()
            ############模型预测#################
#             for key in range(5)
#             predicted_test_labels = new_model.predict(test_images_norm)
#             print(predicted_test_labels.shape)
#             print(predicted_test_labels[88])


#             predicted_test_labels_index = np.argmax(predicted_test_labels[88])

#             print(predicted_test_labels_index)
#             print(test_labels[88])

            #plt.subplot(3,3,3*k+3)
            # plt.figure(3*k+3)
            # plt.figure()
            # plt.imshow(np.squeeze(test_images[88]))
            # plt.show()
            plt.figure(figsize=(10,10))
            for key in range(5):
                number = random.randint(0,10000)
                print("number is "+str(number))
                plt.subplot(1,5,key+1)
                predicted_test_labels = new_model.predict(test_images_norm)
                print(predicted_test_labels.shape)
                print(predicted_test_labels[number])


                predicted_test_labels_index = np.argmax(predicted_test_labels[number])

                print(predicted_test_labels_index)
                print(test_labels[number])

                #plt.subplot(3,3,3*k+3)
                #plt.figure(3*k+3)
                #plt.figure()
                plt.imshow(np.squeeze(test_images[number]))
            plt.show()
print("模型不同参数配置成功")